# 01_deduplicate_order_reviews.ipynb

## 1. Objective
Reads in the raw olist_order_reviews_dataset.csv, removes earlier reviews so each order_id appears only once. The output will be loaded into the database schema.

## 2. Setup

In [1]:
import pandas as pd

## 3. Data Loading

In [2]:
raw_df = pd.read_csv('../data/raw/olist_order_reviews_dataset.csv', 
                 parse_dates=['review_creation_date', 'review_answer_timestamp'])

raw_df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53


## 4. Inspect The Dataframe
Verifies and inspects the duplicate values in the potential primary keys: review_id and order_id

In [3]:
raw_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_id                99224 non-null  object        
 1   order_id                 99224 non-null  object        
 2   review_score             99224 non-null  int64         
 3   review_comment_title     11568 non-null  object        
 4   review_comment_message   40977 non-null  object        
 5   review_creation_date     99224 non-null  datetime64[ns]
 6   review_answer_timestamp  99224 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 27.8 MB


In [4]:
raw_df.nunique()

review_id                  98410
order_id                   98673
review_score                   5
review_comment_title        4527
review_comment_message     36159
review_creation_date         636
review_answer_timestamp    98248
dtype: int64

In [5]:
raw_df[raw_df['order_id'].duplicated(keep=False)]['order_id'].value_counts().head()

order_id
df56136b8031ecd28e200bb18e6ddb2e    3
c88b1d1b157a9999ce368f218a407141    3
03c939fd7fd3b38f8485a0f95798f1f6    3
8e17072ec97ce29f0e1f111e598b0c85    3
169d7e0fd71d624d306f132acd791cbe    2
Name: count, dtype: int64

In [6]:
raw_df[raw_df['order_id'].duplicated(keep=False)].sort_values('review_answer_timestamp').head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
23389,f24e1946d658c13928818f1a73b6d899,b19169c7cb8391fb8561e094d61c531a,5,NaN,Excelente produto.\r\nObrigada,2016-10-27,2016-10-28 12:21:53
41735,f24e1946d658c13928818f1a73b6d899,c4f710df20f7d1500da1aef81a993f65,5,NaN,Excelente produto.\r\nObrigada,2016-10-27,2016-10-28 12:21:53
75532,87159a093663b2a1bb847dbb84879c41,c4f710df20f7d1500da1aef81a993f65,5,NaN,Excelente produto.\r\nVendedor nota 1000,2016-10-29,2016-10-31 20:00:13
71050,87159a093663b2a1bb847dbb84879c41,b19169c7cb8391fb8561e094d61c531a,5,NaN,Excelente produto.\r\nVendedor nota 1000,2016-10-29,2016-10-31 20:00:13
1023,948a6406a9e50a798f9e3c5b3dfced86,b6e5aa946acc4e29e7069510f28a0bce,5,NaN,NaN,2017-01-27,2017-01-27 23:51:32


## 5. Deduplicate by order_id

In [7]:
# Keep only the most recent review for each order
deduped_df = raw_df.sort_values('review_answer_timestamp').drop_duplicates(subset='order_id', keep='last').reset_index(drop=True)
deduped_df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,6916ca4502d6d3bfd39818759d55d536,bfbd0f9bdef84302105ad712db648a6c,1,NaN,nao recebi o produto e nem resposta da empresa,2016-10-06 00:00:00,2016-10-07 18:32:28
1,49f695dffa457eaba90d388a5c37e942,e5215415bb6f76fe3b7cb68103a0d1c0,1,NaN,"PRODUTO NÃO CHEGOU,E JÁ PASSOU O PRAZO DE ENTREGA",2016-10-09 00:00:00,2016-10-11 14:31:29
2,743d98b1a4782f0646898fc915ef002a,e2144124f98f3bf46939bc5183104041,4,NaN,NaN,2016-10-15 00:00:00,2016-10-16 03:20:17
3,53752edb26544dd41c1209f582c9c589,b8b9d7046c083150cb5360b83a8ebb51,5,NaN,O pedido foi entregue antes do prazo pr0metido,2016-10-16 01:00:00,2016-10-16 15:45:11
4,b2d5d8db2a841d27a72e4c06c6212368,9aa3197e4887919fde0307fc23601d7a,4,NaN,Só chegou uma parte do pedido ate agora..,2016-10-15 00:00:00,2016-10-17 21:02:49


## 6. Verify Uniqueness of order_id

In [8]:
deduped_df.shape

(98673, 7)

In [9]:
deduped_df['order_id'].nunique()

98673

## 7. Reorder Columns

In [12]:
# Moves order_id to the front of the table
final_df = deduped_df[['order_id', 'review_id', 'review_score', 'review_comment_title',
                            'review_comment_message', 'review_creation_date', 'review_answer_timestamp']]
final_df.head()

,order_id,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,bfbd0f9bdef84302105ad712db648a6c,6916ca4502d6d3bfd39818759d55d536,1,NaN,nao recebi o produto e nem resposta da empresa,2016-10-06 00:00:00,2016-10-07 18:32:28
1,e5215415bb6f76fe3b7cb68103a0d1c0,49f695dffa457eaba90d388a5c37e942,1,NaN,"PRODUTO NÃO CHEGOU,E JÁ PASSOU O PRAZO DE ENTREGA",2016-10-09 00:00:00,2016-10-11 14:31:29
2,e2144124f98f3bf46939bc5183104041,743d98b1a4782f0646898fc915ef002a,4,NaN,NaN,2016-10-15 00:00:00,2016-10-16 03:20:17
3,b8b9d7046c083150cb5360b83a8ebb51,53752edb26544dd41c1209f582c9c589,5,NaN,O pedido foi entregue antes do prazo pr0metido,2016-10-16 01:00:00,2016-10-16 15:45:11
4,9aa3197e4887919fde0307fc23601d7a,b2d5d8db2a841d27a72e4c06c6212368,4,NaN,Só chegou uma parte do pedido ate agora..,2016-10-15 00:00:00,2016-10-17 21:02:49


## 8. Save New File

In [11]:
final_df.to_csv('../data/interim/deduped_olist_review_orders_dataset')